<a href="https://colab.research.google.com/github/victordotm/TFM/blob/master/Filtrar_Imagenes_Motos_VGG16_Imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
BASE_FOLDER = "/content/drive/My Drive/Colab Notebooks/TFM/"
BASE_FOLDER_filtro = BASE_FOLDER+"Dataset_filtroVGG16_Imagenet/"

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tarfile
import pickle
%matplotlib inline

import cv2
from google.colab.patches import cv2_imshow
import numpy as np

####Path base de datos fotografias a filtrar

In [0]:
BASE_FOLDER_noDup_ecooltra = BASE_FOLDER+"Dataset_noDup/ecooltra/"
BASE_FOLDER_noDup_acciona = BASE_FOLDER+"Dataset_noDup/acciona/"
BASE_FOLDER_noDup_yego = BASE_FOLDER+"Dataset_noDup/yego/"
BASE_FOLDER_noDup_muving = BASE_FOLDER+"Dataset_noDup/muving/"
BASE_FOLDER_noDup_blinkee = BASE_FOLDER+"Dataset_noDup/blinkee/"
BASE_FOLDER_noDup_motos = BASE_FOLDER+"Dataset_noDup/motos/"

####Path base de datos fotografias filtradas

In [0]:
BASE_FOLDER_vgg16Motos_ecooltra = BASE_FOLDER_filtro+"ecooltra/"
BASE_FOLDER_vgg16Motos_acciona = BASE_FOLDER_filtro+"acciona/"
BASE_FOLDER_vgg16Motos_yego = BASE_FOLDER_filtro+"yego/"
BASE_FOLDER_vgg16Motos_muving = BASE_FOLDER_filtro+"muving/"
BASE_FOLDER_vgg16Motos_blinkee = BASE_FOLDER_filtro+"blinkee/"
BASE_FOLDER_vgg16Motos_motos = BASE_FOLDER_filtro+"motos/"

In [0]:
#definimos una función para listar todas las imagenes de una misma compañia de alquiler

def listarImagenes (carpetas, listaImagenes):
  
  for numfolders in range(len(carpetas)):
  
    for r, d, f in os.walk(carpetas[numfolders]):
        for file in f:
                listaImagenes.append(os.path.join(r, file))

  for f in listaImagenes:
      print(f)
  
  return True

In [0]:
#contendrá la lista de ficheros de imágenes de cada compañia de alquiler
files_ecooltra = []
files_yego = []
files_muving = []
files_acciona = []
files_blinkee = []
files_motos = []

In [0]:
folders_ecooltra = []
folders_yego = []
folders_muving = []
folders_acciona = []
folders_blinkee = []
folders_motos = []

folders_ecooltra.append(BASE_FOLDER_noDup_ecooltra)
folders_yego.append(BASE_FOLDER_noDup_yego)
folders_muving.append(BASE_FOLDER_noDup_muving)
folders_acciona.append(BASE_FOLDER_noDup_acciona)
folders_blinkee.append(BASE_FOLDER_noDup_blinkee)
folders_motos.append(BASE_FOLDER_noDup_motos)

#### Listamos la ruta de las imagenes disponibles en la base de datos de origen, para cada compañia de alquiler.

In [0]:
listarImagenes (folders_ecooltra, files_ecooltra)
listarImagenes (folders_acciona, files_acciona)
listarImagenes (folders_yego, files_yego)
listarImagenes (folders_blinkee, files_blinkee)
listarImagenes (folders_muving, files_muving)
listarImagenes (folders_motos, files_motos)

/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image0.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image1.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image2.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image3.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image4.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image5.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image6.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image7.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image8.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image9.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image10.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecooltra/image11.jpg
/content/drive/My Drive/Colab Notebooks/TFM/Dataset_noDup/ecoo

True

### Filtro aplicando red neuronal VGG16 preentrenda con imagenet

In [0]:
from keras.models import load_model
VGG16_pretrained = load_model(BASE_FOLDER+"VGG16_pretrained.h5py")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


#### Obtenemos las etiquetas de las clases de imagenet

In [0]:
with open(BASE_FOLDER+"pickle_all_classes.pickle", "rb") as input_file:
    labels = pickle.load(input_file)

In [0]:
from sklearn import preprocessing

#El valor de labels de la base de datos es string. Lo debemos convertir en numerico para que nuestro modelo neuronal pueda trabajar con él.

le = preprocessing.LabelEncoder()
le.fit(labels)
labels=le.transform(labels)


#### Función para filtrar las imagenes <br>
Pasamos por la red neuronal cada una de las imagnes de la base de datos original.<br>
Guardamos aquellas imagenes cuya etiqueta indicada por la red neuronal es igual a "Motorbike"

In [0]:
#Decorador para calcular_tiempo a la función para la cual queremos calcular el tiempo de ejecución

from time import time
#Función para calcular el tiempo de ejecución
def calcular_tiempo(f): # @calcular_tiempo
   
    def wrapper(*args, **kwargs):        
        inicio = time()       
        resultado = f(*args, **kwargs)       
        tiempo = time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado
    
    return wrapper

In [0]:
@calcular_tiempo
def clasificar_imagenes(files_path, save_folder):
  
  #Contamos el número de decteciones e imagenes guardadas
  num_motobike = 0
  num_image2write = 0
  imagenes_iniciales = 0
  
  for f in files_path:
    
    imagenes_iniciles = imagenes_iniciales+1
    img = cv2.imread(f)
    img = cv2.resize(img,(128,128),3)
    imgInput = np.array(img).reshape((1, 128, 128, 3))

    value_predicted=VGG16_pretrained.predict_classes(imgInput,batch_size=1)
    if le.inverse_transform([value_predicted]) == 'Motorbikes':
      num_motobike = num_motobike +1
      cv2.imwrite(save_folder+"image"+str(num_image2write)+".jpg",img)
      num_image2write = num_image2write +1
  
  print("Motos guardadas en: "+save_folder)
  print("Numero de imagenes guardadas: ")
  print(num_motobike)
  print("Numero imagenes iniciales: ")
  print(imagenes_iniciales)
  porcent_filtro = (num_motobike/imagenes_iniciales)*100
  print("porcentaje filtrado: ")
  print(porcent_filtro)
  return num_motobike

In [0]:
clasificar_imagenes(files_ecooltra, BASE_FOLDER_vgg16Motos_ecooltra)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Motos guardadas en: /content/drive/My Drive/Colab Notebooks/TFM/Dataset_filtroVGG16_Imagenet/ecooltra/
Numero de imagenes guardadas: 
61
Numero imagenes iniciales: 
411
porcentaje filtrado: 
14.841849148418493
Tiempo de ejecución para algoritmo: 76.8345091342926


61

In [0]:
clasificar_imagenes(files_acciona, BASE_FOLDER_vgg16Motos_acciona)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Motos guardadas en: /content/drive/My Drive/Colab Notebooks/TFM/Dataset_filtroVGG16_Imagenet/acciona/
Numero de imagenes guardadas: 
62
Numero imagenes iniciales: 
425
porcentaje filtrado: 
14.588235294117647
Tiempo de ejecución para algoritmo: 198.72851371765137


62

In [0]:
clasificar_imagenes(files_yego, BASE_FOLDER_vgg16Motos_yego)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Motos guardadas en: /content/drive/My Drive/Colab Notebooks/TFM/Dataset_filtroVGG16_Imagenet/yego/
Numero de imagenes guardadas: 
26
Numero imagenes iniciales: 
388
porcentaje filtrado: 
6.701030927835052
Tiempo de ejecución para algoritmo: 156.0465793609619


26

In [0]:
clasificar_imagenes(files_muving, BASE_FOLDER_vgg16Motos_muving)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Motos guardadas en: /content/drive/My Drive/Colab Notebooks/TFM/Dataset_filtroVGG16_Imagenet/muving/
Numero de imagenes guardadas: 
27
Numero imagenes iniciales: 
473
porcentaje filtrado: 
5.708245243128964
Tiempo de ejecución para algoritmo: 200.09170055389404


27

In [0]:
clasificar_imagenes(files_blinkee, BASE_FOLDER_vgg16Motos_blinkee)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Motos guardadas en: /content/drive/My Drive/Colab Notebooks/TFM/Dataset_filtroVGG16_Imagenet/blinkee/
Numero de imagenes guardadas: 
20
Numero imagenes iniciales: 
253
porcentaje filtrado: 
7.905138339920949
Tiempo de ejecución para algoritmo: 104.62871980667114


20

In [0]:
clasificar_imagenes(files_motos, BASE_FOLDER_vgg16Motos_motos)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Motos guardadas en: /content/drive/My Drive/Colab Notebooks/TFM/Dataset_filtroVGG16_Imagenet/motos/
Numero de imagenes guardadas: 
310
Numero imagenes iniciales: 
470
porcentaje filtrado: 
65.95744680851064
Tiempo de ejecución para algoritmo: 234.8030207157135


310